In [49]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import librosa


In [50]:
df_diag = pd.read_csv('Respiratory_Sound_Database/patient_diagnosis.csv', names=['PatientID', 'Diagnosis'])

In [51]:
orig_counts = df_diag['Diagnosis'].value_counts()
print("Original counts:\n", orig_counts)


Original counts:
 Diagnosis
COPD              64
Healthy           26
URTI              14
Bronchiectasis     7
Bronchiolitis      6
Pneumonia          6
LRTI               2
Asthma             1
Name: count, dtype: int64


In [52]:
rare_classes = orig_counts[orig_counts < 5].index.tolist()
if len(rare_classes) > 0:
    df_diag['Diagnosis'] = df_diag['Diagnosis'].replace(rare_classes, 'Other')
    print(f"\nMerged rare classes into 'Other': {rare_classes}")


Merged rare classes into 'Other': ['LRTI', 'Asthma']


In [53]:
patient_labels = df_diag.groupby('PatientID')['Diagnosis'].first().reset_index()

In [54]:
final_counts = patient_labels['Diagnosis'].value_counts()
print("\nFinal patient counts per class:")
print(final_counts)


Final patient counts per class:
Diagnosis
COPD              64
Healthy           26
URTI              14
Bronchiectasis     7
Pneumonia          6
Bronchiolitis      6
Other              3
Name: count, dtype: int64


In [55]:
train_patients, test_patients = train_test_split(
    patient_labels,
    test_size=0.2,
    stratify=patient_labels['Diagnosis'],
    random_state=42
)

In [56]:
# Convert to sets for fast lookup
train_patient_ids = set(train_patients['PatientID'])
test_patient_ids = set(test_patients['PatientID'])

In [57]:
def extract_mfcc_features(audio_path, n_mfcc=13):
    # Load audio (mono, 22050 Hz)
    audio, sr = librosa.load(audio_path, sr=22050)
    
    # If audio too short, pad it
    if len(audio) < sr * 4:  # 4 seconds
        audio = np.pad(audio, (0, sr * 4 - len(audio)), mode='constant')
    else:
        audio = audio[:sr * 4]  # take first 4 sec
    
    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    
    # Use stats: mean, std, min, max → 13 x 4 = 52 features
    mean = np.mean(mfccs, axis=1)
    std = np.std(mfccs, axis=1)
    min_ = np.min(mfccs, axis=1)
    max_ = np.max(mfccs, axis=1)
    
    return np.concatenate([mean, std, min_, max_])


In [58]:
audio_folder = 'Respiratory_Sound_Database/audio_and_txt_files'
audio_files = [f for f in os.listdir(audio_folder) if f.endswith('.wav')]


In [59]:
def get_patient_id(filename):
    return int(filename.split('_')[0])

In [60]:
# Prepare train & test lists
X_train, y_train = [], []
X_test, y_test = [], []


In [61]:
for file in audio_files:
    pid = get_patient_id(file)
    label = df_diag[df_diag['PatientID'] == pid]['Diagnosis'].values[0]
    features = extract_mfcc_features(os.path.join(audio_folder, file))
    
    if pid in train_patient_ids:
        X_train.append(features)
        y_train.append(label)
    elif pid in test_patient_ids:
        X_test.append(features)
        y_test.append(label)

In [62]:
# Convert to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)

In [63]:
# Train model
model = RandomForestClassifier(
    n_estimators=200,
    class_weight='balanced',  # 👈 handles imbalance!
    random_state=42
)

In [64]:
model.fit(X_train, y_train)

,n_estimators,200
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [65]:
y_pred = model.predict(X_test)

In [66]:
# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nDetailed Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8686868686868687

Detailed Report:
                precision    recall  f1-score   support

Bronchiectasis       0.00      0.00      0.00         3
 Bronchiolitis       0.20      0.50      0.29         2
          COPD       0.89      0.99      0.94       171
       Healthy       0.33      0.10      0.15        10
         Other       0.00      0.00      0.00         1
     Pneumonia       0.00      0.00      0.00         4
          URTI       0.00      0.00      0.00         7

      accuracy                           0.87       198
     macro avg       0.20      0.23      0.20       198
  weighted avg       0.79      0.87      0.82       198



/home/gopika/Desktop/lung_disease/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/gopika/Desktop/lung_disease/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/gopika/Desktop/lung_disease/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier,

In [67]:
from audiomentations import Compose, AddGaussianNoise, PitchShift, TimeStretch

augmenter = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.7),
    PitchShift(min_semitones=-1.5, max_semitones=1.5, p=0.7),
    TimeStretch(min_rate=0.9, max_rate=1.1, p=0.7)
])
rare_labels = ['Other']  # add more if needed

In [68]:
X_train, y_train = [], []
X_test, y_test = [], []

for file in audio_files:
    pid = get_patient_id(file)
    label = df_diag[df_diag['PatientID'] == pid]['Diagnosis'].values[0]
    audio_path = os.path.join(audio_folder, file)
    
    # Load original audio
    audio, sr = librosa.load(audio_path, sr=22050)
    if len(audio) < sr * 4:
        audio = np.pad(audio, (0, sr * 4 - len(audio)), mode='constant')
    else:
        audio = audio[:sr * 4]

    # Decide if train or test
    is_train = pid in train_patient_ids

    # Function to extract features from audio array
    def get_features(aud):
        mfccs = librosa.feature.mfcc(y=aud, sr=sr, n_mfcc=13)
        mean = np.mean(mfccs, axis=1)
        std = np.std(mfccs, axis=1)
        min_ = np.min(mfccs, axis=1)
        max_ = np.max(mfccs, axis=1)
        return np.concatenate([mean, std, min_, max_])

    if is_train:
        # Add original sample
        X_train.append(get_features(audio))
        y_train.append(label)
        
        # If rare class, add 3 augmented versions
        if label in rare_labels:
            for _ in range(3):
                aug_audio = augmenter(samples=audio, sample_rate=sr)
                X_train.append(get_features(aug_audio))
                y_train.append(label)
    else:
        # Test: no augmentation!
        X_test.append(get_features(audio))
        y_test.append(label)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

print("Train samples after augmentation:", len(X_train))
print("Class distribution in train:", pd.Series(y_train).value_counts())

Train samples after augmentation: 728
Class distribution in train: COPD              622
Pneumonia          33
Healthy            25
URTI               16
Bronchiectasis     13
Bronchiolitis      11
Other               8
Name: count, dtype: int64


In [69]:
rf = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
print("✅ Random Forest Accuracy:", acc_rf)

✅ Random Forest Accuracy: 0.8686868686868687


In [70]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm = SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42)
svm.fit(X_train, y_train)

# Predict labels
y_pred_labels = svm.predict(X_test)

# Predict probabilities (optional)
y_pred_probs = svm.predict_proba(X_test)

# Accuracy
acc_svm = accuracy_score(y_test, y_pred_labels)
print("✅ SVM Accuracy:", acc_svm)


✅ SVM Accuracy: 0.7373737373737373


In [71]:
# 🚀 CORRECTED SEQUENCE EXTRACTION (Only MFCC sequences for 1D CNN)

# 🔁 RE-EXTRACT MFCCs as sequences (for 1D CNN input)
def extract_mfcc_sequence(audio_path, n_mfcc=13, max_len=100):
    audio, sr = librosa.load(audio_path, sr=22050)
    if len(audio) < sr * 4:
        audio = np.pad(audio, (0, sr * 4 - len(audio)), mode='constant')
    else:
        audio = audio[:sr * 4]
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)  # shape: (13, time)
    mfccs = mfccs.T  # (time, 13)
    if mfccs.shape[0] > max_len:
        mfccs = mfccs[:max_len]
    else:
        pad_width = max_len - mfccs.shape[0]
        mfccs = np.pad(mfccs, ((0, pad_width), (0, 0)), mode='constant')
    return mfccs

# Rebuild X_train_seq, X_test_seq with augmentation for rare classes
X_train_seq, y_train_seq = [], []
X_test_seq, y_test_seq = [], []

for file in audio_files:
    pid = get_patient_id(file)
    label = df_diag[df_diag['PatientID'] == pid]['Diagnosis'].values[0]
    is_train = pid in train_patient_ids
    
    # Load and process original audio
    audio_path = os.path.join(audio_folder, file)
    audio, sr = librosa.load(audio_path, sr=22050)
    if len(audio) < sr * 4:
        audio = np.pad(audio, (0, sr * 4 - len(audio)), mode='constant')
    else:
        audio = audio[:sr * 4]
    
    # Get original sequence
    mfcc_seq = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13).T
    if mfcc_seq.shape[0] > 100:
        mfcc_seq = mfcc_seq[:100]
    else:
        mfcc_seq = np.pad(mfcc_seq, ((0, 100 - mfcc_seq.shape[0]), (0, 0)), mode='constant')
    
    if is_train:
        X_train_seq.append(mfcc_seq)
        y_train_seq.append(label)
        
        # If rare class, add augmented versions
        if label in rare_labels:
            for _ in range(3):
                aug_audio = augmenter(samples=audio, sample_rate=sr)
                mfcc_aug = librosa.feature.mfcc(y=aug_audio, sr=sr, n_mfcc=13).T
                if mfcc_aug.shape[0] > 100:
                    mfcc_aug = mfcc_aug[:100]
                else:
                    mfcc_aug = np.pad(mfcc_aug, ((0, 100 - mfcc_aug.shape[0]), (0, 0)), mode='constant')
                X_train_seq.append(mfcc_aug)
                y_train_seq.append(label)
    else:
        X_test_seq.append(mfcc_seq)
        y_test_seq.append(label)

X_train_seq = np.array(X_train_seq)
X_test_seq = np.array(X_test_seq)
y_train_seq = np.array(y_train_seq)
y_test_seq = np.array(y_test_seq)

print("✅ Sequence data ready!")
print("X_train_seq shape:", X_train_seq.shape)
print("X_test_seq shape:", X_test_seq.shape)

✅ Sequence data ready!
X_train_seq shape: (728, 100, 13)
X_test_seq shape: (198, 100, 13)


In [72]:

# 🚀 NOW RUN THE IMPROVED CNN
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight

# Encode labels to integers
le2 = LabelEncoder()
y_train_enc2 = le2.fit_transform(y_train_seq)  # ← Use y_train_seq
y_test_enc2 = le2.transform(y_test_seq)       # ← Use y_test_seq

# One-hot encode for CNN
num_classes = len(le2.classes_)
y_train_cat2 = to_categorical(y_train_enc2, num_classes)
y_test_cat2 = to_categorical(y_test_enc2, num_classes)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_enc2), y=y_train_enc2)
class_weight_dict = dict(enumerate(class_weights))

# Improved CNN model
model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(100, 13)),
    BatchNormalization(),
    Conv1D(64, kernel_size=5, activation='relu'),
    MaxPooling1D(2),
    Dropout(0.3),

    Conv1D(128, kernel_size=3, activation='relu'),
    BatchNormalization(),
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(2),
    Dropout(0.4),

    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(le2.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callback: stops training if model stops improving
early_stop = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)

# Train the model with CORRECT variables
history = model.fit(
    X_train_seq, y_train_cat2,
    validation_data=(X_test_seq, y_test_cat2),
    epochs=100,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[early_stop],
    verbose=1
)

# Check new accuracy
y_pred_cnn = model.predict(X_test_seq)
y_pred_classes = np.argmax(y_pred_cnn, axis=1)
acc_cnn_new = accuracy_score(y_test_enc2, y_pred_classes)
print("✅ Improved CNN Accuracy:", acc_cnn_new)

/home/gopika/Desktop/lung_disease/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 126ms/step - accuracy: 0.3379 - loss: 3.6424 - val_accuracy: 0.2172 - val_loss: 1.6769
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - accuracy: 0.5000 - loss: 2.0720 - val_accuracy: 0.1364 - val_loss: 1.7993
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.4272 - loss: 1.7152 - val_accuracy: 0.6414 - val_loss: 1.0105
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.5522 - loss: 1.5696 - val_accuracy: 0.6768 - val_loss: 0.9405
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.5165 - loss: 1.6086 - val_accuracy: 0.7121 - val_loss: 0.8805
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 109ms/step - accuracy: 0.5893 - loss: 1.3997 - val_accuracy: 0.6566 - val_loss: 0.8323
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 117ms/step - accuracy: 0.6250 - loss: 1.3967 - val_accuracy: 0.6414 - val_loss: 0.8461
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 134ms/step - accuracy: 0.5632 - loss: 1.4264 - val_accurac

In [73]:
import numpy as np
import librosa
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

def extract_mel_spectrogram(audio_path, n_mels=128, n_time=128):
    audio, sr = librosa.load(audio_path, sr=22050)
    # Pad or trim to fixed length (4 seconds)
    if len(audio) < sr * 4:
        audio = np.pad(audio, (0, sr * 4 - len(audio)), mode='constant')
    else:
        audio = audio[:sr * 4]
    
    # Create mel-spectrogram (no n_time parameter)
    n_fft = 2048
    hop_length = 512  # to get ~128 time steps for 4 sec audio
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)  # Convert to dB
    
    # Now resize to fixed shape (128, 128)
    if mel_spec_db.shape[1] > n_time:
        mel_spec_db = mel_spec_db[:, :n_time]
    else:
        mel_spec_db = np.pad(mel_spec_db, ((0, 0), (0, n_time - mel_spec_db.shape[1])), mode='constant')
    
    # Return shape: (128, 128, 1) - like a grayscale image
    return mel_spec_db.reshape(128, 128, 1)

# Helper function for augmented audio
def extract_mel_spectrogram_from_audio(audio, sr, n_mels=128, n_time=128):
    # Pad or trim audio
    if len(audio) < sr * 4:
        audio = np.pad(audio, (0, sr * 4 - len(audio)), mode='constant')
    else:
        audio = audio[:sr * 4]
    
    # Create mel-spectrogram
    n_fft = 2048
    hop_length = 512
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Resize
    if mel_spec_db.shape[1] > n_time:
        mel_spec_db = mel_spec_db[:, :n_time]
    else:
        mel_spec_db = np.pad(mel_spec_db, ((0, 0), (0, n_time - mel_spec_db.shape[1])), mode='constant')
    
    return mel_spec_db.reshape(128, 128, 1)

# Build 2D spectrogram data
X_train_2d, y_train_2d = [], []
X_test_2d, y_test_2d = [], []

for file in audio_files:
    pid = get_patient_id(file)
    label = df_diag[df_diag['PatientID'] == pid]['Diagnosis'].values[0]
    is_train = pid in train_patient_ids
    
    # Extract spectrogram
    spec = extract_mel_spectrogram(os.path.join(audio_folder, file))
    
    if is_train:
        X_train_2d.append(spec)
        y_train_2d.append(label)
        
        # Augment for rare classes
        if label in rare_labels:
            audio, sr = librosa.load(os.path.join(audio_folder, file), sr=22050)
            if len(audio) < sr * 4:
                audio = np.pad(audio, (0, sr * 4 - len(audio)), mode='constant')
            else:
                audio = audio[:sr * 4]
            
            for _ in range(3):
                aug_audio = augmenter(samples=audio, sample_rate=sr)
                aug_spec = extract_mel_spectrogram_from_audio(aug_audio, sr)
                X_train_2d.append(aug_spec)
                y_train_2d.append(label)
    else:
        X_test_2d.append(spec)
        y_test_2d.append(label)

X_train_2d = np.array(X_train_2d)
X_test_2d = np.array(X_test_2d)
y_train_2d = np.array(y_train_2d)
y_test_2d = np.array(y_test_2d)

print("✅ 2D Spectrogram data ready!")
print("X_train_2d shape:", X_train_2d.shape)
print("X_test_2d shape:", X_test_2d.shape)

✅ 2D Spectrogram data ready!
X_train_2d shape: (728, 128, 128, 1)
X_test_2d shape: (198, 128, 128, 1)


In [74]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight

# Encode labels
le_2d = LabelEncoder()
y_train_enc_2d = le_2d.fit_transform(y_train_2d)
y_test_enc_2d = le_2d.transform(y_test_2d)
y_train_cat_2d = to_categorical(y_train_enc_2d, len(le_2d.classes_))
y_test_cat_2d = to_categorical(y_test_enc_2d, len(le_2d.classes_))

# Compute class weights
class_weights_2d = compute_class_weight('balanced', classes=np.unique(y_train_enc_2d), y=y_train_enc_2d)
class_weight_dict_2d = dict(enumerate(class_weights_2d))

# 2D CNN Model
model_2d = Sequential([
    # Block 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.3),

    # Block 2
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.4),

    # Block 3
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.5),

    # Classifier
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(le_2d.classes_), activation='softmax')
])

model_2d.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callback
early_stop_2d = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)

# Train
history_2d = model_2d.fit(
    X_train_2d, y_train_cat_2d,
    validation_data=(X_test_2d, y_test_cat_2d),
    epochs=100,
    batch_size=32,
    class_weight=class_weight_dict_2d,
    callbacks=[early_stop_2d],
    verbose=1
)

# Evaluate
y_pred_2d = model_2d.predict(X_test_2d)
y_pred_classes_2d = np.argmax(y_pred_2d, axis=1)
acc_2d = accuracy_score(y_test_enc_2d, y_pred_classes_2d)
print("✅ 2D CNN Accuracy:", acc_2d)

/home/gopika/Desktop/lung_disease/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100


2025-10-29 13:21:35.537369: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 47710208 exceeds 10% of free system memory.
2025-10-29 13:21:43.325700: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 65028096 exceeds 10% of free system memory.
2025-10-29 13:21:43.427749: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 65028096 exceeds 10% of free system memory.
2025-10-29 13:21:43.524546: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 65028096 exceeds 10% of free system memory.
2025-10-29 13:21:43.586854: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 62980096 exceeds 10% of free system memory.


23/23 ━━━━━━━━━━━━━━━━━━━━ 139s 6s/step - accuracy: 0.2033 - loss: 2.3373 - val_accuracy: 0.0101 - val_loss: 2.2854
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 130s 6s/step - accuracy: 0.2624 - loss: 1.9057 - val_accuracy: 0.0152 - val_loss: 3.2868
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 135s 6s/step - accuracy: 0.3159 - loss: 1.8752 - val_accuracy: 0.0101 - val_loss: 1.6607
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 136s 6s/step - accuracy: 0.1909 - loss: 1.9534 - val_accuracy: 0.8636 - val_loss: 1.1480
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 136s 6s/step - accuracy: 0.2445 - loss: 1.8133 - val_accuracy: 0.0354 - val_loss: 2.0642
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 146s 6s/step - accuracy: 0.2143 - loss: 1.7356 - val_accuracy: 0.0354 - val_loss: 9.9991
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 195s 6s/step - accuracy: 0.2005 - loss: 1.7764 - val_accuracy: 0.0354 - val_loss: 10.0681
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 135s 6s/step - accuracy: 0.2706 - loss: 1.7283 - val_accuracy: 0.0253 - val_

In [75]:
import numpy as np

# 🔁 Get predictions from ALL 4 models
# Assuming you have:
# - rf (Random Forest model) 
# - svm (SVM model)
# - model (1D CNN model)
# - model_2d (2D CNN model)

# Get individual predictions (PROBABILITIES - not class predictions)
rf_probs = rf.predict_proba(X_test)  # RF probabilities
svm_probs = svm.predict_proba(X_test)  # ❌ Was: svm.predict(X_test) → Wrong! Use predict_proba
cnn1d_probs = model.predict(X_test_seq)  # 1D CNN probabilities (already probabilities)
cnn2d_probs = model_2d.predict(X_test_2d)  # 2D CNN probabilities (already probabilities)

# 🧠 Ensemble: Average the probabilities (simple voting)
print("RF probs shape:", rf_probs.shape)
print("SVM probs shape:", svm_probs.shape)  # ← This was missing
print("1D CNN probs shape:", cnn1d_probs.shape)
print("2D CNN probs shape:", cnn2d_probs.shape)

# Average predictions from ALL 4 models
ensemble_probs = (rf_probs + svm_probs + cnn1d_probs + cnn2d_probs) / 4

# Convert to class predictions
ensemble_pred_classes = np.argmax(ensemble_probs, axis=1)

# 🧪 Evaluate ensemble
# Use the same label encoder as your best model (e.g., le2 from 1D CNN)
ensemble_accuracy = accuracy_score(y_test_enc2, ensemble_pred_classes)
print("✅ Ensemble Accuracy (4 models):", ensemble_accuracy)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step
RF probs shape: (198, 7)
SVM probs shape: (198, 7)
1D CNN probs shape: (198, 7)
2D CNN probs shape: (198, 7)
✅ Ensemble Accuracy (4 models): 0.8636363636363636


In [76]:
print("\n" + "="*40)
print("FINAL ACCURACIES")
print("="*40)
print(f"Random Forest : {acc_rf:.4f}")
print(f"SVM           : {acc_svm:.4f}")
print(f"CNN2          : {acc_2d:.4f}")
print(f"CNN1           : {acc_cnn_new:.4f}")
print("="*40)


FINAL ACCURACIES
Random Forest : 0.8687
SVM           : 0.7374
CNN2          : 0.8636
CNN1           : 0.7980


In [77]:
import joblib
import pickle
from tensorflow.keras.models import load_model

# 🔒 Save Random Forest
joblib.dump(rf, 'best_random_forest_model.pkl')
print("✅ Random Forest saved!")

# 🔒 Save SVM  
joblib.dump(svm, 'svm_model.pkl')
print("✅ SVM saved!")

# 🔒 Save 1D CNN (using .keras format as recommended)
model.save('1d_cnn_model.keras')
print("✅ 1D CNN saved!")

# 🔒 Save 2D CNN (using .keras format as recommended)
model_2d.save('2d_cnn_model.keras')
print("✅ 2D CNN saved!")

# 🔒 Save label encoders (only the ones that exist)
try:
    joblib.dump(le2, 'label_encoder_1dcnn.pkl')  # for 1D CNN
    print("✅ 1D CNN label encoder saved!")
except NameError:
    print("⚠️ le2 not found - 1D CNN encoder not saved")

try:
    joblib.dump(le_2d, 'label_encoder_2dcnn.pkl')  # for 2D CNN
    print("✅ 2D CNN label encoder saved!")
except NameError:
    print("⚠️ le_2d not found - 2D CNN encoder not saved")

# 🔒 Save class names (try to get from existing encoders)
try:
    with open('class_names.pkl', 'wb') as f:
        pickle.dump(list(le2.classes_), f)  # from 1D CNN encoder
    print("✅ Class names saved!")
except NameError:
    print("⚠️ Cannot save class names - encoder not found")

print("\n📁 Models saved successfully!")
print("Recommended files to keep:")
print("- best_random_forest_model.pkl (best model)")
print("- 1d_cnn_model.keras")
print("- 2d_cnn_model.keras")

✅ Random Forest saved!
✅ SVM saved!
✅ 1D CNN saved!
✅ 2D CNN saved!
✅ 1D CNN label encoder saved!
✅ 2D CNN label encoder saved!
✅ Class names saved!

📁 Models saved successfully!
Recommended files to keep:
- best_random_forest_model.pkl (best model)
- 1d_cnn_model.keras
- 2d_cnn_model.keras


In [78]:
import joblib
import pickle
import numpy as np
from tensorflow.keras.models import load_model

# 🔓 Load Random Forest
rf_loaded = joblib.load('best_random_forest_model.pkl')
print("✅ Random Forest loaded!")

# 🔓 Load SVM
svm_loaded = joblib.load('svm_model.pkl')
print("✅ SVM loaded!")

# 🔓 Load 1D CNN
cnn_1d_loaded = load_model('1d_cnn_model.keras')
print("✅ 1D CNN loaded!")

# 🔓 Load 2D CNN
cnn_2d_loaded = load_model('2d_cnn_model.keras')
print("✅ 2D CNN loaded!")

# 🔓 Load label encoders (if they exist)
try:
    le2_loaded = joblib.load('label_encoder_1dcnn.pkl')
    print("✅ 1D CNN label encoder loaded!")
except:
    print("⚠️ 1D CNN label encoder not found")
    le2_loaded = None

try:
    le_2d_loaded = joblib.load('label_encoder_2dcnn.pkl')
    print("✅ 2D CNN label encoder loaded!")
except:
    print("⚠️ 2D CNN label encoder not found")
    le_2d_loaded = None

# 🔓 Load class names
try:
    with open('class_names.pkl', 'rb') as f:
        class_names = pickle.load(f)
    print("✅ Class names loaded!")
    print("Classes:", class_names)
except:
    print("⚠️ Class names not found")
    class_names = None

print("\n✅ All models loaded successfully!")

✅ Random Forest loaded!
✅ SVM loaded!
✅ 1D CNN loaded!
✅ 2D CNN loaded!
✅ 1D CNN label encoder loaded!
✅ 2D CNN label encoder loaded!
✅ Class names loaded!
Classes: [np.str_('Bronchiectasis'), np.str_('Bronchiolitis'), np.str_('COPD'), np.str_('Healthy'), np.str_('Other'), np.str_('Pneumonia'), np.str_('URTI')]

✅ All models loaded successfully!


In [79]:
import numpy as np

# 🔍 Test Random Forest
rf_pred = rf_loaded.predict(X_test)
rf_acc = accuracy_score(y_test, rf_pred)
print(f"✅ Loaded RF Accuracy: {rf_acc:.4f}")

# 🔍 Test SVM
svm_pred = svm_loaded.predict(X_test)
svm_acc = accuracy_score(y_test, svm_pred)
print(f"✅ Loaded SVM Accuracy: {svm_acc:.4f}")

# 🔍 Test 1D CNN
cnn1d_pred_probs = cnn_1d_loaded.predict(X_test_seq)
cnn1d_pred = np.argmax(cnn1d_pred_probs, axis=1)
cnn1d_acc = accuracy_score(y_test_enc2, cnn1d_pred)
print(f"✅ Loaded 1D CNN Accuracy: {cnn1d_acc:.4f}")

# 🔍 Test 2D CNN
cnn2d_pred_probs = cnn_2d_loaded.predict(X_test_2d)
cnn2d_pred = np.argmax(cnn2d_pred_probs, axis=1)
cnn2d_acc = accuracy_score(y_test_enc_2d, cnn2d_pred)
print(f"✅ Loaded 2D CNN Accuracy: {cnn2d_acc:.4f}")

print(f"\n📊 Final Results (Loaded Models):")
print(f"Random Forest: {rf_acc:.4f}")
print(f"SVM: {svm_acc:.4f}")
print(f"1D CNN: {cnn1d_acc:.4f}")
print(f"2D CNN: {cnn2d_acc:.4f}")

# 🏆 Best model
best_model_name = max(
    [('RF', rf_acc), ('SVM', svm_acc), ('1D CNN', cnn1d_acc), ('2D CNN', cnn2d_acc)],
    key=lambda x: x[1]
)
print(f"\n🏆 Best Loaded Model: {best_model_name[0]} with {best_model_name[1]:.4f} accuracy")

✅ Loaded RF Accuracy: 0.8687
✅ Loaded SVM Accuracy: 0.7374
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step
✅ Loaded 1D CNN Accuracy: 0.7980
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step
✅ Loaded 2D CNN Accuracy: 0.8636

📊 Final Results (Loaded Models):
Random Forest: 0.8687
SVM: 0.7374
1D CNN: 0.7980
2D CNN: 0.8636

🏆 Best Loaded Model: RF with 0.8687 accuracy
